In [1]:
# mount drive, to use the uploaded dataset

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# install dependencies

%%capture
!pip install cupy-cuda100

In [3]:
# clone code

!git clone https://github.com/simonedeldeo/pytorch-sepconv.git
%cd pytorch-sepconv

Cloning into 'pytorch-sepconv'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 219 (delta 1), reused 3 (delta 1), pack-reused 211
Receiving objects: 100% (219/219), 14.54 MiB | 7.75 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/pytorch-sepconv


In [0]:
# get dataset

%%capture
!rm -rf val
!cp '/content/drive/My Drive/__progetto int sys/data/val - Star360.zip' '/content/pytorch-sepconv'
!mkdir val
!unzip 'val - Star360.zip' -d val

In [0]:
# preprocess dataset

import os

basepath = '/content/pytorch-sepconv/val'
sub_f = os.listdir(basepath)
sub_f.sort()
for s in sub_f:
  path = os.path.join(basepath, s)
  os.rename(os.path.join(path, '0.jpg'), os.path.join(path, 'frame0.jpg'))
  os.rename(os.path.join(path, '1.jpg'), os.path.join(path, 'frame1.jpg'))
  os.rename(os.path.join(path, '2.jpg'), os.path.join(path, 'frame2.jpg'))

In [6]:
# get pretrained model

!wget --timestamping http://content.sniklaus.com/sepconv/network-l1.pytorch

import torch
from collections import OrderedDict

state_dict = torch.load('network-l1.pytorch')

new_state_dict = OrderedDict()
for k, v in state_dict.items():
  name = 'get_kernel.' + k
  new_state_dict[name] = v

torch.save(new_state_dict, 'l1-mod.pytorch')

--2020-02-22 14:18:42--  http://content.sniklaus.com/sepconv/network-l1.pytorch
Resolving content.sniklaus.com (content.sniklaus.com)... 51.15.58.24
Connecting to content.sniklaus.com (content.sniklaus.com)|51.15.58.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86713338 (83M) [application/octet-stream]
Saving to: ‘network-l1.pytorch’

network-l1.pytorch  100%[===================>]  82.70M  10.4MB/s    in 9.8s    

2020-02-22 14:18:53 (8.45 MB/s) - ‘network-l1.pytorch’ saved [86713338/86713338]



In [0]:
# generate interpolated images

%%capture
!python test.py --input ./val --gt ./val --output ./out --checkpoint ./l1-mod.pytorch

In [8]:
# get stats

import PIL
import PIL.Image
import skimage
import skimage.measure
import numpy
import math
import os

gt = '/frame1.jpg'
interp = '/output.png'

basepath1 = './val/'
basepath2 = './out/'
sub_f = os.listdir(basepath1)
sub_f.sort()
dblPsnr = []
dblSsim = []
dblIE = []
for s in sub_f:
  path1 = os.path.join(basepath1, s)
  path2 = os.path.join(basepath2, s)
  dblPsnr.append(skimage.measure.compare_psnr(im_true=numpy.array(PIL.Image.open(path1 + gt))[:, :, ::-1], im_test=numpy.array(PIL.Image.open(path2 + interp))[:, :, ::-1], data_range=255))
  dblSsim.append(skimage.measure.compare_ssim(X=numpy.array(PIL.Image.open(path1 + gt))[:, :, ::-1], Y=numpy.array(PIL.Image.open(path2 + interp))[:, :, ::-1], data_range=255, multichannel=True))
  dblIE.append(math.sqrt(skimage.measure.compare_mse(numpy.array(PIL.Image.open(path1 + gt))[:, :, ::-1], numpy.array(PIL.Image.open(path2 + interp))[:, :, ::-1])))

print('computed average psnr', numpy.mean(dblPsnr))
print('computed average ssim', numpy.mean(dblSsim))
print('computed average ie', numpy.mean(dblIE))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: DEPRECATED: skimage.measure.compare_psnr has been moved to skimage.metrics.peak_signal_noise_ratio. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: DEPRECATED: skimage.measure.compare_mse has been moved to skimage.metrics.mean_squared_error. It will be removed from skimage.measure in version 0.18.


computed average psnr 27.144348518648993
computed average ssim 0.9152674982777651
computed average ie 14.606128056392144
